In [1]:
from spotdl import Spotdl
from spotdl.utils.config import DEFAULT_CONFIG
from spotdl.download.downloader import Downloader
from yt_dlp import YoutubeDL

sp_client = Spotdl(
    client_id=DEFAULT_CONFIG["client_id"],
    client_secret=DEFAULT_CONFIG["client_secret"],
    user_auth=DEFAULT_CONFIG["user_auth"],
    cache_path=DEFAULT_CONFIG["cache_path"],
    no_cache=True,
    headless=True,
)

yt_client = Downloader(settings={"simple_tui": True})
dl_client = YoutubeDL({'format': 'bestaudio/best'})

Output()

[out#0/wav @ 0x63b3b0c42f00] video:0kB audio:79425kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000096%
size=   79425kB time=00:07:03.60 bitrate=1536.0kbits/s speed= 423x    


In [2]:
from dataclasses import dataclass
from io import BytesIO, TextIOWrapper
import numpy as np
import numpy.typing as npt
import requests
from pyparsing import Any
from os import path, makedirs
import hashlib
import logging
import json
from soundfile import read
from tqdm.notebook import tqdm
import soundfile
from openmxr.convert import convert_opus_to_wav
import pickle

from downloader.DownloadTask import DownloadTask



def mkmd5(string: str):
    return hashlib.md5(string.encode()).hexdigest()

@dataclass
class Metadata:
    title: str
    artist: str | None
    cover: str | None
    album: str | None
    year: str | None
    genre: str | None

@dataclass
class AnalyzedTrack:
    meta: Metadata
    sample_rate: int
    key: int
    waveform: npt.NDArray
    downbeats: list[int]
    best_downbeats: list[int]
    bpm: int  


class Cache:
    name: str
    
    def __init__(self, name: str):
        self.name = name
        self.check_cache_exists()
    
    @staticmethod
    def load(file: TextIOWrapper):
        return json.load(file)

    @staticmethod
    def dump(data, file: TextIOWrapper):
        return json.dump(data, file, ensure_ascii=False)


    def check_cache_exists(self):
        if not path.isdir(path.join("./.cache/", self.name)):
            logging.debug(f"Creating cache {path.join('./.cache/', self.name)}")
            makedirs(path.join("./.cache/", self.name))

    def get_cache_filename(self, cache_key: str) -> str:
        return path.join("./.cache/", self.name, f"{mkmd5(cache_key)}.cache")

    def get(self, key) -> Any | None:
        cache_filename = self.get_cache_filename(key)
        if not path.exists(cache_filename):
            return None
        logging.debug(f"{self.name.upper()}: loading {cache_filename}")
        with open(cache_filename, 'rb') as file:
            return self.load(file)

    def set(self, key, data):
        cache_filename = self.get_cache_filename(key)
        logging.debug(f"{self.name.upper()}: saving {cache_filename}")
        with open(cache_filename, 'wb') as file:
            self.dump(data, file)
            
    
class AudioCache(Cache):
    @staticmethod
    def load(file):
        return tuple(pickle.load(file)) #type: ignore
    
    @staticmethod
    def dump(data, file):
        pickle.dump(data, file) #type: ignore

class Song():
    yt_link: str
    spotify_link: str
    _audio_cache: tuple[int, npt.NDArray] | None = None
    _spotify_cache: dict[str, Any] | None = None
    _yt_cache: dict[str, Any] | None = None
    
    __yt_cache_instance = Cache("yt")
    __spotify_cache_instance = Cache("spotify")
    __audio_cache_instance = AudioCache("audio")
    
    def __init__(self, yt_url, spotify_url):
        self.yt_link = yt_url
        self.spotify_link = spotify_url
        
        if not self._yt_cache:
            if (cached := self.__yt_cache_instance.get(self.yt_link)):
                self._yt_cache = cached
            else:
                self._yt_cache = self._download_yt_meta()
                self.__yt_cache_instance.set(self.yt_link, self._yt_cache)
        if not self._spotify_cache:
            if (cached := self.__spotify_cache_instance.get(self.yt_link)):
                self._spotify_cache = cached
            else:
                self._spotify_cache = self._download_spotify_meta()
                self.__spotify_cache_instance.set(self.yt_link, self._spotify_cache)

        if not self._audio_cache:
            if (cached := self.__audio_cache_instance.get(self.yt_link)):
                self._audio_cache = cached
            else:
                self._audio_cache = self._download_audio()
                self.__audio_cache_instance.set(self.yt_link, self._audio_cache)

        logging.info(f"loaded {self._spotify_cache['name']}")
        
        # print(self._yt_cache["formats"])
        # print(self.yt_link, self.spotify_link, self._spotify_cache, self._yt_cache)
    
    @staticmethod
    def _check_link_valid(url: str):
        req = requests.head(url)
        return req.status_code < 400
            
    
    @classmethod
    def find(cls, spotify_link = None, youtube_link = None, query = None):
        if spotify_link and youtube_link:
            logging.info("Loading custom song config")
            return cls(spotify_link, youtube_link)
        if spotify_link:
            logging.info("Loading from spotify")
            return cls.from_spotify_link(spotify_link)
        if youtube_link:
            logging.info("Loading from youtube")
            return cls.from_yt_link(youtube_link)
        if query:
            logging.info(f"Searching {query}")
            return cls.from_search(query)
    
    @classmethod
    def from_spotify_link(cls, spotify_link: str):
        if (_cache_spotify := cls.__spotify_cache_instance.get(spotify_link)):
            cls._spotify_cache = _cache_spotify
            return cls(_cache_spotify["download_url"], _cache_spotify["url"])

        spotify_song = sp_client.search([spotify_link])[0]
        yt_url = yt_client.search(spotify_song)
        spotify_song.download_url = yt_url
        cls._spotify_cache = spotify_song.json
        cls.__spotify_cache_instance.set(spotify_song.url, spotify_song.json)
        return cls(yt_url, spotify_song.url)
    
    @classmethod
    def from_yt_link(cls, youtube_link: str):
        if (_cache_yt := cls.__yt_cache_instance.get(youtube_link)):    
            cls._youtube_cache = _cache_yt
            return cls(_cache_yt["url"], _cache_yt["CUSTOM__spotify_url"])
        
        yt_song = dl_client.extract_info(youtube_link, download=False)
        cls._yt_cache = yt_song
        
        if yt_song:
            title = f"{yt_song.get('artist', '')} {yt_song.get('track', yt_song['fulltitle'])}"
            spotify_song = sp_client.search([title])[0]
            cls._spotify_cache = spotify_song.json
            spotify_song.download_url = youtube_link
            yt_song["CUSTOM__spotify_url"] = spotify_song.url
            cls.__spotify_cache_instance.set(spotify_song.url, spotify_song.json)
            cls.__yt_cache_instance.set(youtube_link, yt_song)
            return cls(youtube_link, spotify_song.url)
        raise Exception("this url is not valid")
    
    @classmethod
    def from_search(cls, query: str):
        return cls.from_spotify_link(query) # dirty way to search for song
    
    def _download_audio(self) -> tuple[int, npt.NDArray]:
        if not self._yt_cache:
            raise Exception("youtube cache not initialized")
                    
        audio_url = self._yt_cache["url"]
        if not self._check_link_valid(audio_url):
            self._yt_cache = self._download_yt_meta()
            self.__yt_cache_instance.set(self.yt_link, self._yt_cache)
            audio_url = self._yt_cache["url"]
        
        logging.info(f"Downloading {self._yt_cache['fulltitle']}")
        buffer = DownloadTask(audio_url).start()
        logging.info(f"Downloaded {self._yt_cache['fulltitle']}")
        
        logging.info(f"Converting {self._yt_cache['fulltitle']}")
        buffer = convert_opus_to_wav(buffer)
        logging.info(f"Converted {self._yt_cache['fulltitle']}")
        
        y, sr = soundfile.read(BytesIO(buffer))
        return (sr, y)
    
    def _download_spotify_meta(self) -> dict[str, Any]:
        spotify_song = sp_client.search([self.spotify_link])[0]
        return spotify_song.json
    
    def _download_yt_meta(self) -> dict[str, Any]:
        yt_song = dl_client.extract_info(self.yt_link, download=False)
        if yt_song:
            return yt_song
        raise Exception("this url is not valid")
    
    # def analyze(self) -> AnalyzedTrack:
    #     return AnalyzedTrack() 
    
logging.basicConfig(level=logging.INFO)


In [3]:
Song._check_link_valid("https://rr1---sn-2va3vhuxa-f5f6.googlevideo.com/videoplayback?expire=1706415413&ei=1YC1ZanSIuOni9oPqoe3uA0&ip=31.0.144.130&id=o-AN63kiccH-fWO3YZ223WCdaWpjPpnbddB4NHGLC1nlsT&itag=251&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&mh=PJ&mm=31%2C29&mn=sn-2va3vhuxa-f5f6%2Csn-f5f7lnl7&ms=au%2Crdu&mv=m&mvi=1&pl=19&gcr=pl&initcwndbps=446250&vprv=1&svpuc=1&mime=audio%2Fwebm&gir=yes&clen=7251122&dur=423.621&lmt=1626989482374168&mt=1706393598&fvip=3&keepalive=yes&fexp=24007246&c=ANDROID&txp=2311224&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cgcr%2Cvprv%2Csvpuc%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&sig=AJfQdSswRQIhAMowiZQrDWpOOdnpQfYd557tIiT-4s6_V5Vw_ZUosqy2AiAgfPiqvsRhAikKmeQVlj-_URiS12dahIC_i5hMG4RjmA%3D%3D&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AAO5W4owRgIhAKqcJvUpR6IqGEbZFz1tyHMLN_TxPfCxOYpR3k0hfhEOAiEA95DJbQjYRq_2bgRBqltt6aqWvKT_fj_R4KSBiuZJbfU%3D")

False

In [4]:
song = (Song.find(spotify_link="https://open.spotify.com/track/0ddC48e0XVsjoczyIYiGCr"))._download_audio()

INFO:root:Loading from spotify

INFO:root:Downloading No Hay Mañana (Club Mix)

  0%|          | 0.00/7.25M [00:00<?, ?B/s]

INFO:root:Downloaded No Hay Mañana (Club Mix)

INFO:root:Converting No Hay Mañana (Club Mix)

...

  0%|          | 0.00/7.25M [00:00<?, ?B/s]

INFO:root:Downloaded No Hay Mañana (Club Mix)

INFO:root:Converting No Hay Mañana (Club Mix)

ffmpeg started on 2024-01-29 at 21:24:56
Report written to "ffmpeg-20240129-212456.log"
Log level: 48
ffmpeg version n6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.2.1 (GCC) 20230801
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsr

INFO:root:Converted No Hay Mañana (Club Mix)

In [5]:
song

(48000,
 array([[-0.00314331,  0.00991821],
        [ 0.00018311,  0.0138855 ],
        [ 0.00085449,  0.01239014],
        ...,
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]))